# **FIAP Pós-Tech - IA para Devs**

##GRUPO 12 - IA para Devs

```
# Nomes
```



## HACKATHON - DETECÇÃO DE MATERIAIS CORTANTES

A FIAP VisionGuard, empresa de monitoramento de câmeras de segurança, está
analisando a viabilidade de uma nova funcionalidade para otimizar o seu software.

O objetivo da empresa é usar de novas tecnologias para identificar situações atípicas e
que possam colocar em risco a segurança de estabelecimentos e comércios que utilizam
suas câmeras.

Um dos principais desafios da empresa é utilizar Inteligência Artificial para identificar
objetos cortantes (facas, tesouras e similares) e emitir alertas para a central de
segurança.

A empresa tem o objetivo de validar a viabilidade dessa feature, e para isso será
necessário fazer um MVP para detecção supervisionada desses objetos.

### OBJETIVOS

*   Construir ou buscar um dataset contendo imagens de facas, tesouras e outros
objetos cortantes em diferentes condições de ângulo e iluminação.
*   Anotar o dataset para treinar o modelo supervisionado, incluindo imagens
negativas (sem objetos perigosos) para reduzir falsos positivos.
*   Treinar o modelo
*   Desenvolver um sistema de alertas (pode ser e-mail)

### ENTREGÁVEL

*   Documentação detalhando o fluxo utilizado para o desenvolvimento da solução
*   Vídeo de até 15 minutos explicando a solução proposta
*   Link do github do projeto

##O Trabalho

Este script utiliza YOLO para detectar objetos cortantes (facas, machados, tesouras) em vídeos.
Além da detecção, ele processa os vídeos para destacar os objetos encontrados e envia um alerta por e-mail caso haja identificação de itens perigosos.

# Célula 1: Instalação das bibliotecas necessárias

Esta célula instala as bibliotecas necessárias para execução do código.
- `opencv-python-headless`: Versão do OpenCV sem suporte a interface gráfica, ideal para execução em servidores ou ambientes headless.
- `ultralytics`: Biblioteca que contém a implementação do YOLO (You Only Look Once), utilizada para a detecção de objetos.

In [73]:
!pip install opencv-python-headless  # versão headless para evitar problemas de exibição gráfica
!pip install ultralytics

# Célula 2: Montagem do Google Drive

Esta célula monta o Google Drive no ambiente do Colab, permitindo o acesso a arquivos armazenados nele.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Célula 3: Importação das bibliotecas

Importa as bibliotecas necessárias para o funcionamento do sistema:
- `os`: Manipulação de arquivos e diretórios.
- `cv2`: OpenCV para processamento de vídeo.
- `smtplib`: Envio de e-mails via protocolo SMTP.
- `datetime`: Obtenção da data e hora atuais.
- `ultralytics.YOLO`: Modelo de detecção YOLO.
- `email.message.EmailMessage`: Construção de mensagens de e-mail.
- `google.colab.files`: Upload de arquivos no Colab.

In [60]:
import os
import cv2
import smtplib
from datetime import datetime
from ultralytics import YOLO
from email.message import EmailMessage
from google.colab import files  # Para upload do vídeo

# Célula 4: Configuração do e-mail

- Define as configurações para envio de e-mails de alerta
- Cria um diretório chamado `processed_videos` para armazenar os vídeos após o processamento.
Se a pasta já existir, nada será alterado
- Carrega o modelo YOLO treinado para detecção de objetos perigosos
- Cria um conjunto contendo os nomes dos objetos considerados perigosos para detecção

In [ ]:
# Configuração do e-mail
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_SENDER = "securytevisionguard@gmail.com"
EMAIL_PASSWORD = "qihv lyhu ajng gncr"
EMAIL_RECEIVER = "lincoln.soa@gmail.com"

# Pasta para salvar vídeos processados
output_dir = "processed_videos"
os.makedirs(output_dir, exist_ok=True)

# Modelo YOLO para detecção
model = YOLO('/content/best.pt')

# Lista de objetos perigosos a serem detectados
dangerous_objects = {'knife', 'axe', 'scissors'}

# Célula 5: Função para envio de e-mail

Define a função `send_email(detected)`, que envia um e-mail caso objetos perigosos tenham sido detectados no vídeo.
Se um objeto for encontrado, um e-mail de alerta será enviado ao destinatário.

In [70]:
def send_email(detected):
    msg = EmailMessage()
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    if detected:
        msg["Subject"] = "⚠️ Objetos Perigosos Detectados no Vídeo Analisado"
        msg.set_content("Foram detectados objetos cortantes perigosos no vídeo analisado e o resultado desta análise você pode conferir no diretório: processed_videos")
    else:
        print("✅ Nenhum Objeto Perigoso Encontrado no Vídeo")

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("E-mail enviado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar e-mail: {e}")

# Célula 6: Função de processamento de vídeo

A função `process_video(video_path)` abre o vídeo, processa cada frame, detecta objetos perigosos e salva o resultado.
Se um objeto for identificado, ele será destacado no vídeo e um e-mail será enviado.

In [63]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Configura o vídeo de saída
    video_name = os.path.basename(video_path)
    output_video_path = os.path.join(output_dir, f"processed_{video_name}")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    detected_dangerous = False  # Flag para rastrear se objetos perigosos foram encontrados

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].numpy()
                confidence = box.conf[0]
                cls = box.cls[0]
                label = model.names[int(cls)]

                if label in dangerous_objects:
                    detected_dangerous = True
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, f"{label} {confidence:.2f}", (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(frame)  # Salva o frame processado no vídeo de saída

    cap.release()
    out.release()

    print(f"Vídeo processado salvo em: {output_video_path}")

    # Envia e-mail com a detecção final
    send_email(detected_dangerous)

# Célula 7: Definição do vídeo de entrada e processamento

Define o caminho do vídeo a ser analisado e executa a função `process_video()` para iniciar a análise.

In [71]:
video_filename = "/content/video2.mp4"

print("Arquivo de vídeo recebido:", video_filename)
process_video(video_filename)

Arquivo de vídeo recebido: /content/video2.mp4

0: 384x640 1 knife, 1489.2ms
Speed: 3.8ms preprocess, 1489.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 1225.1ms
Speed: 3.2ms preprocess, 1225.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 904.8ms
Speed: 2.5ms preprocess, 904.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 883.7ms
Speed: 3.5ms preprocess, 883.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 axe, 882.8ms
Speed: 2.4ms preprocess, 882.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 866.1ms
Speed: 2.3ms preprocess, 866.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 879.9ms
Speed: 3.1ms preprocess, 879.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 878.0ms
Speed: 3.